## Ejercicio 2: Clasificación de Diabetes con Modelos Supervisados
### Contexto
En esta tarea trabajaremos con el dataset Pima Indians Diabetes (disponible en Kaggle: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database), que contiene datos médicos para predecir si un paciente tiene diabetes (variable objetivo binaria).
El objetivo es construir, evaluar y_2 comparar varios modelos supervisados para clasificación, aplicando preprocesamiento, validación y_2 análisis crítico de resultados, además de entender cómo explicar sus predicciones.

### Parte 1: Exploración y Preprocesamiento
1. Carga el dataset y realiza una exploración inicial:


    * Revisa las primeras filas (head()) y estadísticas básicas (describe()).
    * ¿Cómo se distribuyen los valores de cada columna?
    * ¿Hay valores faltantes o datos anómalos?


In [4]:
# Importar librerías
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
# Cargar datos desde CSV
diabetes = pd.read_csv('diabetes.csv')
diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


### FILAS
La columna de insulin tiene valores de ceros, por lo que nos indica que hay valores faltantes (incluso en pacientes sin insulina se pueden detectar niveles de insulina), lo mismo ocurre para SkinThickness, BMI, BloodPressure, Glucose.

In [6]:
# FILAS #
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Estadísticas descriptivas

In [7]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


2. Aplica las transformaciones necesarias:


    * Limpieza de datos (imputación o eliminación de valores faltantes).    
    * Escalado de características numéricas (StandardScaler, MinMaxScaler, etc.). 
    https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html  
    
    * Codificación si hubiera variables categóricas (en este caso no hay, pero verificar).

In [11]:
diabetes_filtrado = diabetes[(diabetes['Insulin'] != 0) & (diabetes['SkinThickness'] != 0) 
& (diabetes['Glucose'] != 0) & (diabetes['BloodPressure'] != 0) & (diabetes['BMI'] != 0)]
diabetes_filtrado

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
6,3,78,50,32,88,31.0,0.248,26,1
8,2,197,70,45,543,30.5,0.158,53,1
13,1,189,60,23,846,30.1,0.398,59,1
...,...,...,...,...,...,...,...,...,...
753,0,181,88,44,510,43.3,0.222,26,1
755,1,128,88,39,110,36.5,1.057,37,1
760,2,88,58,26,16,28.4,0.766,22,0
763,10,101,76,48,180,32.9,0.171,63,0


In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

diabetesE = StandardScaler().fit_transform(diabetes) # Regresión logistica
diabetes2 = MinMaxScaler().fit_transform(diabetes) # para K-vecinos más cercanos (KNN)
# Arbol de decisión -> no necesita escalado

diabetesE_1 = pd.DataFrame(diabetesE_1, columns=diabetes.columns)
diabetes_2 = pd.DataFrame(diabetes_2, columns=diabetes.columns)

### Parte 2: División de datos
Divide el dataset en tres conjuntos:


* Entrenamiento (60%)


* Validación (20%)


* Test (20%)


In [26]:
from sklearn.model_selection import train_test_split



#Regresion Logistica
X_E = diabetesE_1.drop(columns=['Outcome']).astype(int)
y_E = diabetesE_1['Outcome'].astype(int)
X_E_train, X_E_test, y_E_train, y_E_test = train_test_split(X_E, y_E, test_size=0.2, random_state=42)
X_E_train, X_E_val, y_E_train, y_E_val = train_test_split(X_E_train, y_E_train, test_size=0.25, random_state=42)

#Arbol de decisiones
X = diabetes.drop(columns=['Outcome'])
y = diabetes['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

#KNN
X_2 = diabetes_2.drop(columns=['Outcome'])
y_2 = diabetes_2['Outcome']
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, test_size=0.2, random_state=42)
X_2_train, X_2_val, y_2_train, y_2_val = train_test_split(X_2_train, y_2_train, test_size=0.25, random_state=42)


### Parte 3: Modelos de clasificación
Entrena los siguientes modelos con el conjunto de entrenamiento, usando la validación para ajustar hiperparámetros básicos si quieres:

**1. Regresión logística**


**2. Árbol de decisión** (con profundidad máxima limitada para evitar overfitting)


**3. K-vecinos más cercanos** (KNN) (elige un valor k razonable)
    Explica cómo funciona el algoritmo KNN:
     - ¿Qué aprende el algoritmo KNN? 


In [37]:
#Regresion logistica
from sklearn.linear_model import LogisticRegression

modelo_rl = LogisticRegression().fit(X_E_train, y_E_train)

print(modelo_rl.score(X_E_test, y_E_test))

0.7467532467532467


In [40]:
from sklearn.metrics import confusion_matrix
print("Matriz de confusión (Regresión Logística):")


Matriz de confusión (Regresión Logística):
[0 1]
